# Chronos AI Wealth Management Architecture
Core Input: Real Transaction Dataset Structure
We assume a structure like this per user:

{
  "account_no": "1234567890",
  "starting_balance": 5000,
  "ending_balance": 6200,
  "total_deposited": 3000,
  "total_withdrawn": 1800,
  "average_monthly_balance": 5600,
  "monthly_summary": [
    {"month": "2025-05", "income": 3000, "expenses": 1800}
  ],
  "category_breakdown": [
    {"category": "Groceries", "amount": 600},
    {"category": "Rent", "amount": 1000},
    {"category": "Entertainment", "amount": 400}
  ],
  "transactions": [
    {
      "date": "2025-05-01",
      "amount": -100,
      "description": "Supermarket",
      "category": "Groceries",
      "type": "debit"
    }
  ]
}

In [1]:
#https://github.com/Fawadkhanse/ai-fintech-agent-api/blob/master/README.md

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import requests
from rapidfuzz import fuzz
import re
import json
from dotenv import load_dotenv
import os
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np
import logging
import time
import random  
import re
import string



c:\Users\temesgen.gebreabzgi\Downloads\repos\ch\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Expected info per transaction

In [ ]:
{
  "user_id": "U12345",
  "transactions": [
    {
      "timestamp": "2025-06-10T14:45:00Z",
      "amount": -15.99,
      "currency": "USD",
      "transaction_type": "debit",
      "merchant_name": "Starbucks",
      "merchant_category": "Coffee Shops",
      "description": "Starbucks NY",
      "location": "New York, NY",
      "balance_post_txn": 984.01
    },
    ...
  ]
}


In [2]:

from typing import List, Dict
from datetime import datetime
from collections import defaultdict
import json
import statistics

# Sample transaction type
Transaction = Dict[str, any]

# 1. Analysis Agent - Extract financial insights from raw transactions
class AnalysisAgent:
    def __init__(self, user_id: str, transactions: List[Transaction]):
        self.user_id = user_id
        self.transactions = transactions

    def analyze(self) -> Dict:
        total_income = 0
        total_expense = 0
        categories = defaultdict(float)
        balance_history = []

        for txn in self.transactions:
            amt = txn['amount']
            cat = txn.get('merchant_category', 'Unknown')
            categories[cat] += abs(amt)
            balance_history.append(txn['balance_post_txn'])

            if amt > 0:
                total_income += amt
            else:
                total_expense += abs(amt)

        average_balance = statistics.mean(balance_history) if balance_history else 0

        return {
            'user_id': self.user_id,
            'total_income': round(total_income, 2),
            'total_expense': round(total_expense, 2),
            'category_breakdown': dict(categories),
            'average_balance': round(average_balance, 2)
        }


# 2. Budgeting Agent - Generate monthly budget recommendations
class BudgetingAgent:
    def __init__(self, analysis_result: Dict):
        self.data = analysis_result

    def recommend_budget(self) -> Dict:
        income = self.data['total_income']
        essentials = 0.5 * income
        flexible = 0.3 * income
        savings = 0.2 * income

        overspend = [cat for cat, amt in self.data['category_breakdown'].items() if amt > 0.15 * income]

        return {
            'budget_plan': {
                'essentials': round(essentials, 2),
                'flex_spending': round(flexible, 2),
                'savings': round(savings, 2),
            },
            'overspending_flags': overspend,
            'savings_tips': [
                "Automate your savings",
                "Track non-essential expenses",
                "Review subscriptions"
            ]
        }


# 3. Investment Advisor Agent - Suggest where to invest leftover funds
class InvestmentAdvisorAgent:
    def __init__(self, analysis_result: Dict):
        self.data = analysis_result

    def suggest_investments(self) -> Dict:
        avg_balance = self.data['average_balance']
        reserve = 0.25 * avg_balance
        investable_amount = avg_balance - reserve

        recommendations = [
            {'type': 'Index Fund ETF', 'suggested_amount': 0.5 * investable_amount},
            {'type': 'Crypto (Stablecoin yield)', 'suggested_amount': 0.25 * investable_amount},
            {'type': 'High-Yield Savings', 'suggested_amount': 0.25 * investable_amount}
        ]

        return {
            'investable_funds': round(investable_amount, 2),
            'recommendations': [
                {**r, 'suggested_amount': round(r['suggested_amount'], 2)} for r in recommendations
            ]
        }


# Example usage
if __name__ == '__main__':
    from pprint import pprint

    transactions = [
        {
            "timestamp": "2025-06-10T14:45:00Z",
            "amount": -15.99,
            "currency": "USD",
            "transaction_type": "debit",
            "merchant_name": "Starbucks",
            "merchant_category": "Coffee Shops",
            "description": "Starbucks NY",
            "location": "New York, NY",
            "balance_post_txn": 984.01
        },
        # Add more transactions here
    ]

    user_id = "U12345"
    analysis = AnalysisAgent(user_id, transactions).analyze()
    pprint(analysis)

    budget = BudgetingAgent(analysis).recommend_budget()
    pprint(budget)

    investment = InvestmentAdvisorAgent(analysis).suggest_investments()
    pprint(investment)


{'average_balance': 984.01,
 'category_breakdown': {'Coffee Shops': 15.99},
 'total_expense': 15.99,
 'total_income': 0,
 'user_id': 'U12345'}
{'budget_plan': {'essentials': 0.0, 'flex_spending': 0.0, 'savings': 0.0},
 'overspending_flags': ['Coffee Shops'],
 'savings_tips': ['Automate your savings',
                  'Track non-essential expenses',
                  'Review subscriptions']}
{'investable_funds': 738.01,
 'recommendations': [{'suggested_amount': 369.0, 'type': 'Index Fund ETF'},
                     {'suggested_amount': 184.5,
                      'type': 'Crypto (Stablecoin yield)'},
                     {'suggested_amount': 184.5, 'type': 'High-Yield Savings'}]}


In [3]:
print(analysis)

{'user_id': 'U12345', 'total_income': 0, 'total_expense': 15.99, 'category_breakdown': {'Coffee Shops': 15.99}, 'average_balance': 984.01}
